In [2]:

from run_pipe import get_n_best_res
flag = True
genes_u = ['HIF1A', 'APOL1', 'YAP1', 'SOD1', 'SNCA', 'IRF4', 'KRAS', 'KLKB1', 'SNHG14', 'DGAT2', 'IRF5', 'HTRA1',
           'MYH7', 'MALAT1', 'HSD17B13']

tp = 24
n=12
genes_lst  = ['MALAT1']
res = get_n_best_res(genes_lst,n,tp)
print(res['MALAT1'])


pickle exist
                  Sequence  sense_start  sense_length  sense_start_from_end  \
1894  CATTTTCGTCTGCGTTTAGT         1894            20                  6935   
1903  TAATCTTTCCATTTTCGTCT         1903            20                  6926   
4275  TTTACTTTTTCCATTCTAAG         4275            20                  4554   
2285  CATTTTTTCAGTGCTATTTT         2285            20                  6544   
5040  ATAATCTTTTCTGCCTTTAC         5040            20                  3789   
2155  TTGTTTCTATCTTCCAATTT         2155            20                  6674   
2157  TCTTGTTTCTATCTTCCAAT         2157            20                  6672   
1893  ATTTTCGTCTGCGTTTAGTA         1893            20                  6936   
2005  TATCTTCTCTATTCTTTTCT         2005            20                  6824   
440   TACACTGCTCTGGGTCTGCT          440            20                  8389   
2274  TGCTATTTTATCCAATTTTT         2274            20                  6555   
2007  CCTATCTTCTCTATTCTTTT         2007

In [2]:
from utils import create_gene_to_data
genes_lst  = ['MALAT1']

gene_to_data = create_gene_to_data(genes_lst)

pickle exist


In [3]:
print(gene_to_data['MALAT1'])

LocusInfo:
  exons: [Seq('GACGCAGCCCCACCGGTTGCGCAGTCCCTCCCCGCCCCCGCTCTCCCCTCCGCA...AAG'), Seq('GTGGTAAACTATACCTACTGTCCCTCAAGAGAACACAAGAAGTGCTTTAAGAG'), Seq('GCGGCGGAAGGTGATCGAATTCCGGTGATGCGAGTTGTTCTCCGTCTATAAATA...CAG'), Seq('GGCAAATATTGGCAATTAGTTGGCAGTGGCCTGTTACGGTTGGGATTGGTGGGG...CAA'), Seq('ACTTTTTTCAGATAACATCTTCTGAGTCATAACCAGCCTGGCAGTATGATGGCC...AAA'), Seq('GGGGATTCTTCTCTAATCTTTCAGAAACTTTGTCTGCGAACACTCTTTAATGGA...AAG'), Seq('TTTCCAAAGTTTGCATGTTAACTTTAAATGCTTACAATCTTAGAGTGGTAGGCA...CTA'), Seq('AGGCATTTCATCCTTCATGAAGCCATTCAGGATTTTGAATTGCATATGAGTGCT...CTA'), Seq('ACTTCACAGAGAATGCAGTTGTCTTGACTTCAG'), Seq('ACTTCACAGAGAATGCAGTTGTCTTGACTTCAGGTCTGTCTGTTCTGTTGGCAA...AAG'), Seq('ATGCAACCTTAAAATCAGTGACAAGAAACATT'), Seq('CGTATGCGAAGGGCCAGAGAAGCCAGACCCAGTAAGAAAAAATAGCCTATTTAC...AAG'), Seq('CACCGAAGGCTTAAAGTAGGACAACCATGGAGCCTTCCTGTGGCAGGAGAGACA...AAG'), Seq('CGAAGGCTTAAAGTAGGACAACCATGGAGCCTTCCTGTGGCAGGAGAGACAACA...AAG'), Seq('CTTTTCCAGAAGCCTGTTAAAAGCAAGGTCTCCCCACAAGCAACTTCTCTGCCA...AAG'), Seq('A

TypeError: __str__ returned non-string (type NoneType)

In [ ]:
from utils import *
#plot_res(model,X,y,df_copy)



In [ ]:
# Missing: CAI_score_global_CDS, 'sense_avg_accessibility', RNaseH1_Krel_score_R7_krel, Modification_min_distance_to_3prime

In [ ]:
from features.mod_features import compute_mod_min_distance_to_3prime

# generate MOE 20-mers
for gene, df in dfs.items():
    df.loc[:, 'Modification_min_distance_to_3prime'] = compute_mod_min_distance_to_3prime('MMMMMddddddddddMMMMM')

In [ ]:
import pandas as pd
import glob

# Load all batch files from the new output folder
files = sorted(glob.glob(f"out/batch_*.csv"))
df_all = pd.concat([pd.read_csv(f) for f in files], axis=0)


In [ ]:
df_all

In [ ]:
malat_scores = model.predict(df_all[selected_features])


In [ ]:
from asodesigner.util import get_antisense

# Assuming you already have get_antisense(seq: str) -> str defined
# get_antisense, for some reason numba doesn't work well

df_all["score"] = malat_scores

df_all["sense"] = df_all[SEQUENCE].astype(str).str.translate(tbl).str[::-1]
(
    df_all.assign(score=malat_scores)
    .sort_values("score", ascending=False)   # sort by score
    .to_csv("malat_scores_model2.csv", index=False)
)


In [ ]:
df_sorted = df_all.sort_values('score', ascending=False)


In [ ]:
# pip install requests
import math, time, threading, urllib.parse, requests
from concurrent.futures import ThreadPoolExecutor, as_completed

UA = {"User-Agent": "python-requests gggenome/greedy"}

def _ggg_hits_leq_json(seq, k, db="hg38", timeout=60, retries=2):
    """Count hits with <=k mismatches via GGGenome JSON; fallback to CSV if needed."""
    s = str(seq).upper().replace("U", "T")
    q = urllib.parse.quote(s)
    url_json = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.json"
    url_csv  = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.csv?download"

    for attempt in range(retries + 1):
        try:
            r = requests.get(url_json, headers=UA, timeout=timeout)
            r.raise_for_status()
            try:
                data = r.json()
            except ValueError:
                raise RuntimeError("JSON parse failed")
            if isinstance(data, list):
                return len(data)
            if isinstance(data, dict):
                if "results" in data and isinstance(data["results"], list): return len(data["results"])
                if "hits" in data and isinstance(data["hits"], list):       return len(data["hits"])
                return sum(len(v) for v in data.values() if isinstance(v, list))
            return 0
        except Exception:
            # greedy CSV fallback
            try:
                r2 = requests.get(url_csv, headers=UA, timeout=timeout)
                r2.raise_for_status()
                return sum(1 for ln in r2.text.splitlines() if ln and not ln.startswith("#"))
            except Exception:
                if attempt < retries:
                    continue
                return 0
    return 0

def _d123_for_sequence(seq, db="hg38"):
    s = str(seq).upper().replace("U", "T")
    if not s:
        return (s, 0, 0, 0)
    L = len(s)
    k_allowed = max(0, math.floor(0.25 * L))  # GGGenome cap
    k0 = _ggg_hits_leq_json(s, 0, db=db)
    k1 = _ggg_hits_leq_json(s, 1, db=db) if k_allowed >= 1 else 0
    k2 = _ggg_hits_leq_json(s, 2, db=db) if k_allowed >= 2 else 0
    k3 = _ggg_hits_leq_json(s, 3, db=db) if k_allowed >= 3 else 0
    d1 = max(0, k1 - k0)
    d2 = max(0, k2 - k1)
    d3 = max(0, k3 - k2)
    return (s, d1, d2, d3)

def add_gggenome_d123(main_df, seq_col="SEQUENCE", db="hg38", *, max_workers=32, print_every=10):
    seqs = (main_df[seq_col].astype(str).str.upper().str.replace("U", "T", regex=False))
    uniq = seqs.dropna().unique().tolist()
    N = len(uniq)
    print(f"[GGG] Unique sequences: {N} | db={db} | workers={max_workers}")

    cache = {}
    lock = threading.Lock()
    t0 = time.perf_counter()
    errs = 0

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(_d123_for_sequence, s, db): s for s in uniq}
        done = 0
        for fut in as_completed(futs):
            s = futs[fut]
            try:
                s_key, d1, d2, d3 = fut.result()
            except Exception:
                d1 = d2 = d3 = 0
                with lock:
                    errs += 1
            with lock:
                cache[s] = (d1, d2, d3)
                done += 1
                if (done == 1) or (done % print_every == 0) or (done == N):
                    elapsed = time.perf_counter() - t0
                    rps = done / elapsed if elapsed > 0 else 0.0
                    print(f"[GGG] {done}/{N} cached | ~{rps:.1f} seq/s | errors={errs}")

    main_df["ggg_d1"] = seqs.map(lambda s: cache.get(s, (0, 0, 0))[0])
    main_df["ggg_d2"] = seqs.map(lambda s: cache.get(s, (0, 0, 0))[1])
    main_df["ggg_d3"] = seqs.map(lambda s: cache.get(s, (0, 0, 0))[2])

    print(f"[GGG] Finished in {time.perf_counter() - t0:.1f}s. Added columns: ggg_d1, ggg_d2, ggg_d3")
    return main_df


# --- usage ---
# main_df = main_df[main_df[SENSE_START] != -1]
result = add_gggenome_d123(df_sorted[:100], seq_col='sense', db="hg38", max_workers=100, print_every=2)

In [ ]:
# Show up to 200 characters per column
pd.set_option("display.max_colwidth", 200)

# Show more columns across the screen
pd.set_option("display.max_columns", None)

# Optionally, widen the console display
pd.set_option("display.width", 200)

print(result[:30][[SEQUENCE, 'sense_start', 'ggg_d1', 'ggg_d2', 'at_skew', 'gc_content', 'on_target_fold_openness_normalized40_15', 'sense_avg_accessibility', 'RNaseH1_Krel_score_R7_krel']])


In [ ]:
df_sorted.tail(30)[['on_target_fold_openness_normalized40_15']]

In [ ]:
# Now let's generate LNA GFP sequences

In [ ]:
# GFP_IN_YEAST = (
#     'ATGGTtAGtAAaGGaGAaGAGTTgTTCACaGGaGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCCTGACCTACGGCGTGCAGTGCTTCAGCCGCTACCCCGACCACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCCATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTGAGCACCCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAGGGTGCTGGGGCAggtacCCCTAAAGATCCAGCCAAACCTCCGGCCAcGGCACAAGTTGTGGGATGGCCACCGGTGAGATCATACCGGAAGAACGTGATGGTTTCCTGCCAAAAATCAAGCGGTGGCCCGGAGGCGGCGGCGTTCGTGAAGTAA'
#     .upper())

GFP_IN_HUMAN = (
"ctttttcgcaacgggtttgccgccagaacacaggaccggtgccaccatggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagcccaagaaaaagcggaaagtgggatccggcgcaacaaacttctctctgctgaaacaagccggagatgtcgaagagaatcctggaccgaccgagtacaagcccacggtgcgcctcgccacccgcgacgacgtccccagggccgtacgcaccctcgccgccgcgttcgccgactaccccgccacgcgccacaccgtcgatccggaccgccacatcgagcgggtcaccgagctgcaagaactcttcctcacgcgcgtcgggctcgacatcggcaaggtgtgggtcgcggacgacggcgccgcggtggcggtctggaccacgccggagagcgtcgaagcgggggcggtgttcgccgagatcggcccgcgcatggccgagttgagcggttcccggctggccgcgcagcaacagatggaaggcctcctggcgccgcaccggcccaaggagcccgcgtggttcctggccaccgtcggagtctcgcccgaccaccagggcaagggtctgggcagcgccgtcgtgctccccggagtggaggcggccgagcgcgccggggtgcccgccttcctggagacctccgcgccccgcaacctccccttctacgagcggctcggcttcaccgtcaccgccgacgtcgaggtgcccgaaggaccgcgcacctggtgcatgacccgcaagcccggtgcctgaacgcgttaagtcgacaatcaacctctggattacaaaatttgtgaaagattgactggtattcttaactatgttgctccttttacgctatgtggatacgctgctttaatgcctttgtatcatgctattgcttcccgtatggctttcattttctcctccttgtataaatcctggttgctgtctctttatgaggagttgtggcccgttgtcaggcaacgtggcgtggtgtgcactgtgtttgctgacgcaacccccactggttggggcattgccaccacctgtcagctcctttccgggactttcgctttccccctccctattgccacggcggaactcatcgccgcctgccttgcccgctgctggacaggggctcggctgttgggcactgacaattccgtggtgttgtcggggaaatcatcgtcctttccttggctgctcgcctgtgttgccacctggattctgcgcgggacgtccttctgctacgtcccttcggccctcaatccagcggaccttccttcccgcggcctgctgccggctctgcggcctcttccgcgtcttcgccttcgccctcagacgagtcggatctccctttgggccgcctccccgcgtcgactttaagaccaatgacttacaaggcagctgtagatcttagccactttttaaaagaaaaggggggactggaagggctaattcactcccaacgaagacaagatctgctttttgcttgtactgggtctctctggttagaccagatctgagcctgggagctctctggctaactagggaacccactgcttaagcctcaataaagcttgccttgagtgcttcaagtagtgtgtgcccgtctgttgtgtgactctggtaactagagatccctcagacccttttagtcagtgtggaaaatctctagcagggcccgtttaaacccgctgatcagcctcgactgtgccttctagttgccagccatctgttgtttgcccctcccccgtgccttccttgaccctggaaggtgccactcccactgtcctttcctaataaa".upper()
)
# GFP + Degron x <= 842
# NLS 843 <= x<= 893
# 3UTR 1615 <= x <= 1848


In [ ]:
GFP_IN_HUMAN[633:633+16]


In [ ]:
GFP_IN_HUMAN[679:679+16]

In [ ]:
from asodesigner.util import get_antisense
import pandas as pd

# GFP_YEAST_END = len(GFP_IN_YEAST) - 3
GFP_HUMAN_END = len(GFP_IN_HUMAN)
LNA_SIZE = 16

def get_init_df(target_mrna, end):
    candidates = []
    sense_starts = []
    sense_lengths = []
    sense_starts_from_end = []

    for i in range(0, len(target_mrna) - (LNA_SIZE - 1)):
        target = target_mrna[i: i + LNA_SIZE]
        candidates.append(get_antisense(str(target)))
        sense_starts.append(i)
        sense_lengths.append(LNA_SIZE)
        sense_starts_from_end.append(end - i)
    df = pd.DataFrame(
        {SEQUENCE: candidates, SENSE_START: sense_starts,
         SENSE_LENGTH: sense_lengths, "sense_start_from_end": sense_starts_from_end})
    return df


# df_yeast = get_init_df(GFP_IN_YEAST, GFP_YEAST_END)
df_human = get_init_df(GFP_IN_HUMAN, GFP_HUMAN_END)

In [ ]:
from asodesigner.util import get_antisense
import pandas as pd

# df_yeast[CANONICAL_GENE] = 'YEAST_GFP'
df_human[CANONICAL_GENE] = 'HUMAN_GFP'
dataframes = [df_human]

# df_yeast['sense_exon'] = 1
df_human['sense_exon'] = 1
# df_yeast['sense_intron'] = 0
df_human['sense_intron'] = 0
# df_yeast['sense_utr'] = 0
df_human['sense_utr'] = [1 if sense_start > 842 else 0 for sense_start in df_human[SENSE_START]]


In [ ]:
%reload_ext autoreload
%autoreload 2

from asodesigner.process_utils import LocusInfo
from scripts.data_genertion.data_handling import get_populate_fold

genes_u = ['YEAST_GFP', 'HUMAN_GFP', 'MALAT1']
gene_to_data = {'YEAST_GFP': LocusInfo(), 'HUMAN_GFP': LocusInfo(), 'MALAT1': LocusInfo()}
# gene_to_data['YEAST_GFP'].full_mrna = GFP_IN_YEAST
gene_to_data['HUMAN_GFP'].full_mrna = GFP_IN_HUMAN

In [ ]:
from scripts.data_genertion.data_handling import populate_features

dfs = dict()
# dfs['YEAST_GFP'] = df_yeast
dfs['HUMAN_GFP'] = df_human
SEQUENCES = {}
for target in ['HUMAN_GFP']:
# for target in ['YEAST_GFP', 'HUMAN_GFP']:
    SEQUENCES[target] = gene_to_data[target].full_mrna

for gene, df in dfs.items():
    df[TREATMENT_PERIOD] = 24  # keep constant for all
    df[VOLUME] = 1000  # keep constant for all
    df['log_volume'] = np.log(df[VOLUME])
    df['normalized_start'] = df[SENSE_START] / len(SEQUENCES[gene])
    df['normalized_sense_start_from_end'] = df['sense_start_from_end'] / len(SEQUENCES[gene])
    easy_to_populate = ['at_skew', 'gc_content', 'gc_content_3_prime_5', 'gc_skew', 'hairpin_score',
                        'homooligo_count', 'internal_fold', 'nucleotide_diversity', 'self_energy', 'stop_codon_count',
                        'at_rich_region_score', 'poly_pyrimidine_stretch']
    populate_features(df, easy_to_populate)

In [ ]:
def get_populate_fold(df, genes_u, gene_to_data, fold_variants=[(40, 15)]):
    from asodesigner.fold import calculate_energies, get_weighted_energy
    from asodesigner.util import get_antisense

    all_data_human_gene_premrna_no_nan = df.copy()

    # Comment out the long cases for quick running
    for (window_size, step_size) in fold_variants:

        on_target_fold = 'on_target_fold_openness' + str(window_size) + '_' + str(step_size)
        on_target_fold_normalized = 'on_target_fold_openness_normalized' + str(window_size) + '_' + str(step_size)
        all_data_human_gene_premrna_no_nan[on_target_fold] = np.zeros_like(all_data_human_gene_premrna_no_nan[SEQUENCE],
                                                                           dtype=np.float64)
        all_data_human_gene_premrna_no_nan[on_target_fold_normalized] = np.zeros_like(
            all_data_human_gene_premrna_no_nan[SEQUENCE], dtype=np.float64)

        for gene in genes_u:

            target = gene_to_data[gene].full_mrna
            gene_rows = all_data_human_gene_premrna_no_nan[all_data_human_gene_premrna_no_nan[CANONICAL_GENE] == gene]
            energies = calculate_energies(str(target), step_size, window_size)

            for index, row in gene_rows.iterrows():
                antisense = row[SEQUENCE]
                sense = get_antisense(antisense)
                l = row[SENSE_LENGTH]
                sense_start = row[SENSE_START]
                mean_fold = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                mean_fold_end = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                mean_fold_start = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                if mean_fold > 100:
                    print(energies)
                    print("Weird: ", mean_fold)
                    print("Sense_start ", sense_start)
                    print("Sense_length ", l)
                    print("Gene: ", gene)
                    mean_fold = 0
                all_data_human_gene_premrna_no_nan.loc[index, on_target_fold] = mean_fold
                all_data_human_gene_premrna_no_nan.loc[index, on_target_fold_normalized] = mean_fold / l
    return all_data_human_gene_premrna_no_nan

In [ ]:
from scripts.data_genertion.data_handling import get_populate_fold

# dfs['YEAST_GFP'] = df_yeast
dfs['HUMAN_GFP'] = df_human

for gene, df in dfs.items():
    fold_variants = [(40, 15)]

    df = get_populate_fold(df, ['YEAST_GFP', 'HUMAN_GFP'], gene_to_data, fold_variants=fold_variants)
    dfs[gene] = df

In [ ]:
from hybridization.hybridization_features import get_exp_psrna_hybridization

for gene, df in dfs.items():
    df.loc[:, 'exp_ps_hybr'] = [
        get_exp_psrna_hybridization(antisense.replace('T', 'U'), temp=37) for
        antisense in df[SEQUENCE]]

In [ ]:
from features.mod_features import compute_mod_min_distance_to_3prime

# generate LNA 16-mers
for gene, df in dfs.items():
    df.loc[:, 'Modification_min_distance_to_3prime'] = compute_mod_min_distance_to_3prime('LLLddddddddddLLL')

In [14]:
# Missing: CAI_score_global_CDS, 'sense_avg_accessibility', RNaseH1_Krel_score_R7_krel, Modification_min_distance_to_3prime

from yehuda_code.Folding_Functions import get_sense_with_flanks
from yehuda_code.access_calculator import AccessCalculator

FLANK_SIZE = 120
ACCESS_SIZE = 13
SEED_SIZE = 13
SEED_SIZES = [SEED_SIZE * m for m in range(1, 4)]
ACCESS_WIN_SIZE = 80


def compute_sense_accessibility(row, flank_size, access_win_size, seed_sizes, access_size, min_gc=0, max_gc=100,
                                gc_ranges=1):
    try:
        # Skip invalid rows
        if row['sense_start'] == -1 or pd.isna(row['sense_with_flank_120nt']) or row['sense_with_flank_120nt'] == "":
            return None

        seq = row[f'sense_with_flank_{flank_size}nt']
        sense_start = row['sense_start']
        sense_length = row['sense_length']

        # Calculate accessibility
        df_access = AccessCalculator.calc(
            seq, access_size,
            min_gc, max_gc, gc_ranges,
            access_win_size, seed_sizes
        )

        flank_start = max(0, sense_start - flank_size)
        sense_start_in_flank = sense_start - flank_start
        sense_end_in_flank = sense_start_in_flank + sense_length

        if 0 <= sense_start_in_flank < len(df_access) and sense_end_in_flank <= len(df_access):
            values = df_access['avg_access'].iloc[sense_start_in_flank:sense_end_in_flank].dropna()
            return values.mean() if not values.empty else None
        else:
            return None

    except Exception as e:
        print(f"Error at row {row.name} | seq start: {row['sense_start']} | error: {e}")
        return None


for gene, df in dfs.items():
    FLANKED_SENSE_COL = f'sense_with_flank_{FLANK_SIZE}nt'

    val = gene_to_data[gene].full_mrna
    df['pre_mrna_sequence'] = [val] * len(df)

    # Create new column with flanked sequences
    df[FLANKED_SENSE_COL] = df.apply(
        lambda row: get_sense_with_flanks(
            row['pre_mrna_sequence'],
            row['sense_start'],
            row['sense_length'],
            flank_size=FLANK_SIZE
        ) if row['sense_start'] != -1 else "",  # Handle cases where sense was not found
        axis=1
    )

    batch_size = 500
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch = df.iloc[start_idx:end_idx].copy()

        print(f"Processing rows {start_idx} to {end_idx}...")

        batch['sense_avg_accessibility'] = batch.apply(
            compute_sense_accessibility,
            axis=1,
            flank_size=FLANK_SIZE,
            access_win_size=ACCESS_WIN_SIZE,
            seed_sizes=SEED_SIZES,
            access_size=ACCESS_SIZE,
        )

        # Save batch to the new folder
        batch.to_csv(f"out/{gene}/batch_{start_idx}_{end_idx}.csv", index=False)


Processing rows 0 to 500...


OSError: Cannot save file into a non-existent directory: 'out/MALAT1'

In [ ]:
import pandas as pd
import glob

# Load all batch files from the new output folder
files = sorted(glob.glob(f"out/HUMAN_GFP/batch_*.csv"))
df_human = pd.concat([pd.read_csv(f) for f in files], axis=0)

# files = sorted(glob.glob(f"out/YEAST_GFP/batch_*.csv"))
# df_yeast = pd.concat([pd.read_csv(f) for f in files], axis=0)

In [ ]:

from asodesigner.features.RNaseH_features import rnaseh1_dict, compute_rnaseh1_score

best_window_start_krel = {
    'R4a_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 3, 18: 2, 19: 4, 20: 3, 21: 0, 22: 0, 25: 0},
    'R4b_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 3, 19: 1, 20: 3, 21: 0, 22: 0, 25: 0},
    'R7_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 3, 17: 2, 18: 4, 19: 6, 20: 4, 21: 0, 22: 0, 25: 0},
}

for exp in ['R4a_krel', 'R4b_krel', 'R7_krel']:
    weights = rnaseh1_dict(exp)


    def score_row(row):
        length = len(row['Sequence'])
        pos = best_window_start_krel.get(exp, {}).get(length, 0)
        return compute_rnaseh1_score(row['Sequence'], weights, window_start=pos)

    col_name = f"RNaseH1_Krel_score_{exp}"

    # Yeast should have similar motifs, perhaps
    df_human[col_name] = df_human.apply(score_row, axis=1)
    # df_yeast[col_name] = df_yeast.apply(score_row, axis=1)

RNaseH1_Krel_features_best = [f"RNaseH1_Krel_score_{exp}" for exp in ['R4a_krel', 'R4b_krel', 'R7_krel']]


In [ ]:
# Number is probably accurate enough
df_human.loc[:, 'CAI_score_global_CDS'] = 0.5752 # in frame =1
# df_yeast.loc[:, 'CAI_score_global_CDS'] = 0.5590591814785562

In [ ]:
df_human_scores = model.predict(df_human[selected_features])

# Assuming you already have get_antisense(seq: str) -> str defined
# get_antisense, for some reason numba doesn't work well
tbl = str.maketrans("ACGTUacgtuNn", "TGCAAtgcaaNn")

df_human["score"] = df_human_scores

df_human["sense"] = df_human[SEQUENCE].astype(str).str.translate(tbl).str[::-1]
(
    df_human.assign(score=df_human_scores)
    .sort_values("score", ascending=False)  # sort by score
    .to_csv("df_human_scores_model2.csv", index=False)
)

df_sorted = df_human.sort_values('score', ascending=False)

In [ ]:
# pip install requests
import math, time, threading, urllib.parse, requests
from concurrent.futures import ThreadPoolExecutor, as_completed

UA = {"User-Agent": "python-requests gggenome/greedy"}

def _ggg_hits_leq_json(seq, k, db="hg38", timeout=180, retries=2):
    """Count hits with <=k mismatches via GGGenome JSON; fallback to CSV if needed."""
    s = str(seq).upper().replace("U", "T")
    q = urllib.parse.quote(s)
    url_json = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.json"
    url_csv  = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.csv?download"

    for attempt in range(retries + 1):
        try:
            r = requests.get(url_json, headers=UA, timeout=timeout)
            r.raise_for_status()
            try:
                data = r.json()
            except ValueError:
                raise RuntimeError("JSON parse failed")
            if isinstance(data, list):
                return len(data)
            if isinstance(data, dict):
                if "results" in data and isinstance(data["results"], list): return len(data["results"])
                if "hits" in data and isinstance(data["hits"], list):       return len(data["hits"])
                return sum(len(v) for v in data.values() if isinstance(v, list))
            return 0
        except Exception:
            # greedy CSV fallback
            try:
                r2 = requests.get(url_csv, headers=UA, timeout=timeout)
                r2.raise_for_status()
                return sum(1 for ln in r2.text.splitlines() if ln and not ln.startswith("#"))
            except Exception:
                if attempt < retries:
                    continue
                return 0
    return 0

def _d123_for_sequence(seq, db="hg38"):
    s = str(seq).upper().replace("U", "T")
    if not s:
        return (s, 0, 0, 0)
    L = len(s)
    k_allowed = max(0, math.floor(0.25 * L))  # GGGenome cap
    k0 = _ggg_hits_leq_json(s, 0, db=db)
    k1 = _ggg_hits_leq_json(s, 1, db=db) if k_allowed >= 1 else 0
    # k2 = _ggg_hits_leq_json(s, 2, db=db) if k_allowed >= 2 else 0
    # k3 = _ggg_hits_leq_json(s, 3, db=db) if k_allowed >= 3 else 0
    d1 = max(0, k1 - k0)
    # d2 = max(0, k2 - k1)
    # d3 = max(0, k3 - k2)
    return (s, d1, 0, 0, k0)

cache = {}
def add_gggenome_d123(main_df, seq_col="SEQUENCE", db="hg38", *, max_workers=32, print_every=10):
    seqs = (main_df[seq_col].astype(str).str.upper().str.replace("U", "T", regex=False))
    uniq = seqs.dropna().unique().tolist()
    N = len(uniq)
    print(f"[GGG] Unique sequences: {N} | db={db} | workers={max_workers}")

    global cache
    cache = {}
    lock = threading.Lock()
    t0 = time.perf_counter()
    errs = 0

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(_d123_for_sequence, s, db): s for s in uniq}
        done = 0
        for fut in as_completed(futs):
            s = futs[fut]
            try:
                s_key, d1, d2, d3, d0 = fut.result()
            except Exception:
                d0 = d1 = d2 = d3 = 0
                with lock:
                    errs += 1
            with lock:
                cache[s] = (s, d1, d2, d3, d0)
                done += 1
                if (done == 1) or (done % print_every == 0) or (done == N):
                    elapsed = time.perf_counter() - t0
                    rps = done / elapsed if elapsed > 0 else 0.0
                    print(f"[GGG] {done}/{N} cached | ~{rps:.1f} seq/s | errors={errs}")

    main_df["ggg_d1"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[1])
    main_df["ggg_d0"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[4])
    # main_df["ggg_d2"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[2])
    # main_df["ggg_d3"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[3])


    print(f"[GGG] Finished in {time.perf_counter() - t0:.1f}s. Added columns: ggg_d1, ggg_d2, ggg_d3")
    return main_df


# --- usage ---
# main_df = main_df[main_df[SENSE_START] != -1]
result = add_gggenome_d123(df_sorted[:500], seq_col='sense', db="hg38", max_workers=1, print_every=2)

In [ ]:
result.insert(0, "row_number", range(1, len(result) + 1))

In [ ]:
result[result['ggg_d0'] ==0].head(50)[[SEQUENCE, 'row_number', 'sense_start', 'ggg_d0', 'ggg_d1', 'RNaseH1_Krel_score_R7_krel', 'at_skew', 'gc_content', 'on_target_fold_openness_normalized40_15', 'sense_avg_accessibility']]

In [ ]:
GFP_IN_HUMAN[633:633 + 16]

In [ ]:
GFP_IN_HUMAN[679 :679 + 16]


In [ ]:
# 1. Insert row number (1,2,3,...)
result = result.copy()
# result.insert(0, "row_number", range(1, len(result) + 1))

# 2. Filter on ggg_d1 < 50
filtered = result[result["ggg_d1"] < 500]

# 3. Print selected columns
print(filtered[[
    "row_number", SEQUENCE, "sense_start", "ggg_d1", "at_skew", "gc_content",
    "on_target_fold_openness_normalized40_15",
    "sense_avg_accessibility", "RNaseH1_Krel_score_R7_krel"
]].head(60))


In [ ]:
# Appendix for CAI

In [ ]:
from math import prod

# Yeast codon usage table (S. cerevisiae, from Codon Usage Database)
# frequency per thousand, normalized to relative adaptiveness (w values)
# For simplicity, store as preferred-codon relative adaptiveness
yeast_codon_pref = {
    'TTT':0.61,'TTC':1.0,'TTA':0.19,'TTG':0.45,'CTT':0.10,'CTC':0.08,'CTA':0.07,'CTG':0.09,
    'ATT':0.36,'ATC':1.0,'ATA':0.12,'ATG':1.0,
    'GTT':0.47,'GTC':0.15,'GTA':0.14,'GTG':0.25,
    'TCT':0.55,'TCC':0.54,'TCA':0.37,'TCG':0.17,'AGT':0.39,'AGC':1.0,
    'CCT':0.61,'CCC':0.21,'CCA':0.63,'CCG':0.15,
    'ACT':0.60,'ACC':1.0,'ACA':0.36,'ACG':0.13,
    'GCT':0.82,'GCC':0.49,'GCA':0.58,'GCG':0.13,
    'TAT':0.57,'TAC':1.0,'TAA':0,'TAG':0,'TGA':0,
    'CAT':0.57,'CAC':1.0,
    'CAA':0.34,'CAG':1.0,
    'AAT':0.53,'AAC':1.0,
    'AAA':0.44,'AAG':1.0,
    'GAT':0.46,'GAC':1.0,
    'GAA':0.38,'GAG':1.0,
    'TGT':0.44,'TGC':1.0,'TGG':1.0,
    'CGT':0.56,'CGC':0.65,'CGA':0.14,'CGG':0.14,'AGA':0.05,'AGG':0.02,
    'GGT':0.53,'GGC':0.49,'GGA':1.0,'GGG':0.27
}

# Input sequence (from user)
seq = ("ATGGTtAGtAAaGGaGAaGAGTTgTTCACaGGaGTGGTGCCCATCCTGGTCGAGCTGGACGGCGACGTAAACGGCCACAAGTTCAGCGTGTCCGGCGAGGGCGAGGGCGATGCCACCTACGGCAAGCTGACCCTGAAGTTCATCTGCACCACCGGCAAGCTGCCCGTGCCCTGGCCCACCCTCGTGACCACCCTGACCTACGGCGTGCAGTGCTTCAGCCGCTACCCCGACCACATGAAGCAGCACGACTTCTTCAAGTCCGCCATGCCCGAAGGCTACGTCCAGGAGCGCACCATCTTCTTCAAGGACGACGGCAACTACAAGACCCGCGCCGAGGTGAAGTTCGAGGGCGACACCCTGGTGAACCGCATCGAGCTGAAGGGCATCGACTTCAAGGAGGACGGCAACATCCTGGGGCACAAGCTGGAGTACAACTACAACAGCCACAACGTCTATATCATGGCCGACAAGCAGAAGAACGGCATCAAGGTGAACTTCAAGATCCGCCACAACATCGAGGACGGCAGCGTGCAGCTCGCCGACCACTACCAGCAGAACACCCCCATCGGCGACGGCCCCGTGCTGCTGCCCGACAACCACTACCTGAGCACCCAGTCCGCCCTGAGCAAAGACCCCAACGAGAAGCGCGATCACATGGTCCTGCTGGAGTTCGTGACCGCCGCCGGGATCACTCTCGGCATGGACGAGCTGTACAAGGGTGCTGGGGCAggtacCCCTAAAGATCCAGCCAAACCTCCGGCCAcGGCACAAGTTGTGGGATGGCCACCGGTGAGATCATACCGGAAGAACGTGATGGTTTCCTGCCAAAAATCAAGCGGTGGCCCGGAGGCGGCGGCGTTCGTGAAGTAA").upper()

# Split into codons
codons = [seq[i:i+3] for i in range(0, len(seq), 3)]
# Drop stop codon and any incomplete codons at the end
valid_codons = [c for c in codons if c in yeast_codon_pref and yeast_codon_pref[c] > 0]

# Calculate weights
weights = [yeast_codon_pref[c] for c in valid_codons]

# Compute CAI
cai = prod(weights) ** (1/len(weights))
cai


In [ ]:
from math import prod

# Human codon usage table (Homo sapiens, whole genome, from Kazusa)
# Values are normalized as relative adaptiveness (w)
# Frequencies per 1000 codons (rounded from Kazusa)
human_freqs = {
    'TTT':17.6,'TTC':20.3,'TTA':7.7,'TTG':12.9,'CTT':13.2,'CTC':19.6,'CTA':7.2,'CTG':39.6,
    'ATT':16.0,'ATC':20.8,'ATA':7.5,'ATG':22.0,
    'GTT':10.8,'GTC':14.6,'GTA':7.1,'GTG':28.1,
    'TCT':15.2,'TCC':17.6,'TCA':12.2,'TCG':4.5,'AGT':11.9,'AGC':19.5,
    'CCT':17.5,'CCC':19.8,'CCA':16.9,'CCG':6.9,
    'ACT':13.1,'ACC':21.2,'ACA':15.1,'ACG':6.1,
    'GCT':18.2,'GCC':27.7,'GCA':15.8,'GCG':7.4,
    'TAT':12.0,'TAC':15.6,'TAA':1.0,'TAG':0.8,'TGA':1.6,
    'CAT':10.9,'CAC':15.1,
    'CAA':12.3,'CAG':34.2,
    'AAT':17.0,'AAC':19.1,
    'AAA':24.4,'AAG':32.5,
    'GAT':22.3,'GAC':26.0,
    'GAA':29.0,'GAG':39.6,
    'TGT':10.8,'TGC':12.1,'TGG':13.2,
    'CGT':4.7,'CGC':10.5,'CGA':6.2,'CGG':11.4,'AGA':12.1,'AGG':11.5,
    'GGT':10.8,'GGC':22.2,'GGA':16.5,'GGG':16.5
}

# Normalize to get relative adaptiveness (w)
human_w = {}
for codon in human_freqs:
    aa_group = [c for c in human_freqs if c[0] == codon[0] or True]  # lazy grouping
# Let's just hard-code amino acid codon groupings
aa_codons = {
    'F':['TTT','TTC'],
    'L':['TTA','TTG','CTT','CTC','CTA','CTG'],
    'I':['ATT','ATC','ATA'],
    'M':['ATG'],
    'V':['GTT','GTC','GTA','GTG'],
    'S':['TCT','TCC','TCA','TCG','AGT','AGC'],
    'P':['CCT','CCC','CCA','CCG'],
    'T':['ACT','ACC','ACA','ACG'],
    'A':['GCT','GCC','GCA','GCG'],
    'Y':['TAT','TAC'],
    'H':['CAT','CAC'],
    'Q':['CAA','CAG'],
    'N':['AAT','AAC'],
    'K':['AAA','AAG'],
    'D':['GAT','GAC'],
    'E':['GAA','GAG'],
    'C':['TGT','TGC'],
    'W':['TGG'],
    'R':['CGT','CGC','CGA','CGG','AGA','AGG'],
    'G':['GGT','GGC','GGA','GGG'],
    'STOP':['TAA','TAG','TGA']
}
for aa, codon_list in aa_codons.items():
    maxf = max(human_freqs[c] for c in codon_list)
    for c in codon_list:
        if human_freqs[c] > 0:
            human_w[c] = human_freqs[c]/maxf
        else:
            human_w[c] = 0

# Input sequence (human GFP variant)
seq_human = ("atggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtgtccggcgagggcgagggcgatgccacctacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccctgacctacggcgtgcagtgcttcagccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatcttcttcaaggacgacggcaactacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgagctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaactacaacagccacaacgtctatatcatggccgacaagcagaagaacggcatcaaggtgaacttcaagatccgccacaacatcgaggacggcagcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagcacccagtccgccctgagcaaagaccccaacgagaagcgcgatcacatggtcctgctggagttcgtgaccgccgccgggatcactctcggcatggacgagctgtacaagaagcttagccatggcttcccgccggaggtggaggagcaggatgatggcacgctgcccatgtcttgtgcccaggagagcgggatggaccgtcaccctgcagcctgtgcttctgctaggatcaatgtgaagcgacctgccgccacaaagaaggctggacaggctaagaagaagaaatgaggatcccgcgcgcgcatatgttaattaaccaactgcatggggatccacgcgttaagtcgacaatcaacctctggattacaaaatttgtgaaagattgactggtattcttaactatgttgctccttttacgctatgtggatacgctgctttaatgcctttgtatcatgctattgcttcccgtatggctttcattttctcctccttgtataaatcctggttgctgtctctttatgaggagttgtggcccgttgtcaggcaacgtggcgtggtgtgcactgtgtttgctgacgcaacccccactggttggggcattgccaccacctgtcagctcctttccgggactttcgctttccccctccctattgccacggcggaactcatcgccgcctgccttgcccgctgctggacaggggctcggctgttgggcactgacaattccgtggtgttgtcggggaaatcatcgtcctttccttggctgctcgcctgtgttgccacctggattctgcgcgggacgtccttctgctacgtcccttcggccctcaatccagcggaccttccttcccgcggcctgctgccggctctgcggcctcttccgcgtcttcgccttcgccctcagacgagtcggatctccctttgggccgcctccccgcgtcgactttaagaccaatgacttacaaggcagctgtagatcttagccactttttaaaagaaaaggggggactggaagggctaattcactcccaacgaagacaagatctgctttttgcttgtactgggtctctctggttagaccagatctgagcctgggagctctctggctaactagggaacccactgcttaagcctcaataaagcttgccttgagtgcttcaagtagtgtgtgcccgtctgttgtgtgactctggtaactagagatccctcagacccttttagtcagtgtggaaaatctctagcagtacgtatagtagttcatgtcatcttattattcagtatttataacttgcaaagaaatgaatatcagagagtgagagg")

# Split into codons
codons_h = [seq_human[i:i+3].upper() for i in range(0, len(seq_human), 3)]
valid_codons_h = [c for c in codons_h if c in human_w and human_w[c] > 0]

weights_h = [human_w[c] for c in valid_codons_h]
cai_h = prod(weights_h) ** (1/len(weights_h))
cai_h


In [ ]:
from asodesigner.read_human_genome import get_locus_to_data_dict
import pickle
from asodesigner.consts import CACHE_DIR

# Adding RB1 and TP53BP1
genes_u = ['HIF1A', 'APOL1', 'YAP1', 'SOD1', 'SNCA', 'IRF4', 'KRAS', 'KLKB1', 'SNHG14', 'DGAT2', 'IRF5', 'HTRA1',
           'MYH7', 'MALAT1', 'HSD17B13', 'PRMT5', 'MAT2A', 'RIOK1', 'RB1', 'TP53BP1']
cache_path = CACHE_DIR / 'gene_to_data_simple_cache.pickle'
# if not cache_path.exists():
if True:
    gene_to_data = get_locus_to_data_dict(include_introns=True, gene_subset=genes_u)
    with open(cache_path, 'wb') as f:
        pickle.dump(gene_to_data, f)
else:
    with open(cache_path, 'rb') as f:
        gene_to_data = pickle.load(f)

In [ ]:
SL_PARTNERS = ['RB1', 'TP53BP1']
SEQUENCES = {}
for partner in SL_PARTNERS:
    SEQUENCES[partner] = gene_to_data[partner].full_mrna

In [ ]:
from asodesigner.util import get_antisense
import pandas as pd

# GFP_YEAST_END = len(GFP_IN_YEAST) - 3
GFP_HUMAN_END = len(GFP_IN_HUMAN)
LNA_SIZE = 16

def get_init_df(target_mrna, end, aso_sizes=[LNA_SIZE]):
    candidates = []
    sense_starts = []
    sense_lengths = []
    sense_starts_from_end = []

    for aso_size in aso_sizes:
        for i in range(0, len(target_mrna) - (aso_size - 1)):
            target = target_mrna[i: i + aso_size]
            candidates.append(get_antisense(str(target)))
            sense_starts.append(i)
            sense_lengths.append(aso_size)
            sense_starts_from_end.append(end - i)
    df = pd.DataFrame(
        {SEQUENCE: candidates, SENSE_START: sense_starts,
         SENSE_LENGTH: sense_lengths, "sense_start_from_end": sense_starts_from_end})
    return df

dfs = dict()
# df_yeast = get_init_df(GFP_IN_YEAST, GFP_YEAST_END)
for partner in SL_PARTNERS:
    dfs[partner] = get_init_df(gene_to_data[partner].full_mrna, len(gene_to_data[partner].full_mrna), aso_sizes=[16, 20])

In [ ]:
from scripts.data_genertion.data_handling import get_populated_df_with_structure_features

for gene, df in dfs.items():
    df[CELL_LINE_ORGANISM] = 'human'
    df[INHIBITION] = 0
    df[CANONICAL_GENE] = gene
    df = get_populated_df_with_structure_features(df, genes_u, gene_to_data)

In [ ]:
from scripts.data_genertion.data_handling import populate_features

for gene, df in dfs.items():
    df[TREATMENT_PERIOD] = 24  # keep constant for all
    df[VOLUME] = 1000  # keep constant for all
    df['log_volume'] = np.log(df[VOLUME])
    df['normalized_start'] = df[SENSE_START] / len(SEQUENCES[gene])
    df['normalized_sense_start_from_end'] = df['sense_start_from_end'] / len(SEQUENCES[gene])
    easy_to_populate = ['at_skew', 'gc_content', 'gc_content_3_prime_5', 'gc_skew', 'hairpin_score',
                        'homooligo_count', 'internal_fold', 'nucleotide_diversity', 'self_energy', 'stop_codon_count',
                        'at_rich_region_score', 'poly_pyrimidine_stretch']
    populate_features(df, easy_to_populate)

In [ ]:
def get_populate_fold(df, genes_u, gene_to_data, fold_variants=[(40, 15)]):
    from asodesigner.fold import calculate_energies, get_weighted_energy
    from asodesigner.util import get_antisense

    all_data_human_gene_premrna_no_nan = df.copy()

    # Comment out the long cases for quick running
    for (window_size, step_size) in fold_variants:

        on_target_fold = 'on_target_fold_openness' + str(window_size) + '_' + str(step_size)
        on_target_fold_normalized = 'on_target_fold_openness_normalized' + str(window_size) + '_' + str(step_size)
        all_data_human_gene_premrna_no_nan[on_target_fold] = np.zeros_like(all_data_human_gene_premrna_no_nan[SEQUENCE],
                                                                           dtype=np.float64)
        all_data_human_gene_premrna_no_nan[on_target_fold_normalized] = np.zeros_like(
            all_data_human_gene_premrna_no_nan[SEQUENCE], dtype=np.float64)

        for gene in genes_u:

            target = gene_to_data[gene].full_mrna
            gene_rows = all_data_human_gene_premrna_no_nan[all_data_human_gene_premrna_no_nan[CANONICAL_GENE] == gene]
            energies = calculate_energies(str(target), step_size, window_size)

            for index, row in gene_rows.iterrows():
                antisense = row[SEQUENCE]
                sense = get_antisense(antisense)
                l = row[SENSE_LENGTH]
                sense_start = row[SENSE_START]
                mean_fold = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                mean_fold_end = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                mean_fold_start = get_weighted_energy(sense_start, l, step_size, energies, window_size)
                if mean_fold > 100:
                    print(energies)
                    print("Weird: ", mean_fold)
                    print("Sense_start ", sense_start)
                    print("Sense_length ", l)
                    print("Gene: ", gene)
                    mean_fold = 0
                all_data_human_gene_premrna_no_nan.loc[index, on_target_fold] = mean_fold
                all_data_human_gene_premrna_no_nan.loc[index, on_target_fold_normalized] = mean_fold / l
    return all_data_human_gene_premrna_no_nan

In [ ]:
from scripts.data_genertion.data_handling import get_populate_fold

for gene, df in dfs.items():
    fold_variants = [(40, 15)]
    df = get_populate_fold(df, SL_PARTNERS, gene_to_data, fold_variants=fold_variants)
    dfs[gene] = df

In [ ]:
from hybridization.hybridization_features import get_exp_psrna_hybridization

for gene, df in dfs.items():
    df.loc[:, 'exp_ps_hybr'] = [
        get_exp_psrna_hybridization(antisense.replace('T', 'U'), temp=37) for
        antisense in df[SEQUENCE]]

In [ ]:
# Missing: CAI_score_global_CDS, 'sense_avg_accessibility', RNaseH1_Krel_score_R7_krel, Modification_min_distance_to_3prime
from features.mod_features import compute_mod_min_distance_to_3prime

for gene, df in dfs.items():
    if df[SEQUENCE].str.len().eq(20).any():
        df.loc[df[SEQUENCE].str.len() == 20, "Modification_min_distance_to_3prime"] = \
            compute_mod_min_distance_to_3prime("MMMMMddddddddddMMMMM")

    if df[SEQUENCE].str.len().eq(16).any():
        df.loc[df[SEQUENCE].str.len() == 16, "Modification_min_distance_to_3prime"] = \
            compute_mod_min_distance_to_3prime("LLLddddddddddLLL")

In [ ]:
from yehuda_code.Folding_Functions import get_sense_with_flanks
from yehuda_code.access_calculator import AccessCalculator

FLANK_SIZE = 120
ACCESS_SIZE = 13
SEED_SIZE = 13
SEED_SIZES = [SEED_SIZE * m for m in range(1, 4)]
ACCESS_WIN_SIZE = 80

def compute_sense_accessibility(row, flank_size, access_win_size, seed_sizes, access_size, min_gc=0, max_gc=100,
                                gc_ranges=1):
    try:
        # Skip invalid rows
        if row['sense_start'] == -1 or pd.isna(row['sense_with_flank_120nt']) or row['sense_with_flank_120nt'] == "":
            return None

        seq = row[f'sense_with_flank_{flank_size}nt']
        sense_start = row['sense_start']
        sense_length = row['sense_length']

        # Calculate accessibility
        df_access = AccessCalculator.calc(
            seq, access_size,
            min_gc, max_gc, gc_ranges,
            access_win_size, seed_sizes
        )

        flank_start = max(0, sense_start - flank_size)
        sense_start_in_flank = sense_start - flank_start
        sense_end_in_flank = sense_start_in_flank + sense_length

        if 0 <= sense_start_in_flank < len(df_access) and sense_end_in_flank <= len(df_access):
            values = df_access['avg_access'].iloc[sense_start_in_flank:sense_end_in_flank].dropna()
            return values.mean() if not values.empty else None
        else:
            return None

    except Exception as e:
        print(f"Error at row {row.name} | seq start: {row['sense_start']} | error: {e}")
        return None


for gene, df in dfs.items():
    FLANKED_SENSE_COL = f'sense_with_flank_{FLANK_SIZE}nt'

    val = gene_to_data[gene].full_mrna
    df['pre_mrna_sequence'] = [val] * len(df)

    # Create new column with flanked sequences
    df[FLANKED_SENSE_COL] = df.apply(
        lambda row: get_sense_with_flanks(
            row['pre_mrna_sequence'],
            row['sense_start'],
            row['sense_length'],
            flank_size=FLANK_SIZE
        ) if row['sense_start'] != -1 else "",  # Handle cases where sense was not found
        axis=1
    )

    batch_size = 500
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch = df.iloc[start_idx:end_idx].copy()

        print(f"Processing rows {start_idx} to {end_idx}...")

        batch['sense_avg_accessibility'] = batch.apply(
            compute_sense_accessibility,
            axis=1,
            flank_size=FLANK_SIZE,
            access_win_size=ACCESS_WIN_SIZE,
            seed_sizes=SEED_SIZES,
            access_size=ACCESS_SIZE,
        )

        # Save batch to the new folder
        batch.to_csv(f"out/{gene}/batch_{start_idx}_{end_idx}.csv", index=False)


In [ ]:
import pandas as pd
import glob

files = sorted(glob.glob(f"out/RB1/batch_*.csv"))
df_rb1_accessibility = pd.concat([pd.read_csv(f) for f in files], axis=0)

In [ ]:
df_rb1 = df_rb1_accessibility

In [ ]:
files = sorted(glob.glob(f"out/TP53BP1/batch_*.csv"))
df_tp53bp1_accessibility = pd.concat([pd.read_csv(f) for f in files], axis=0)

In [ ]:
df_tp53bp1 = df_tp53bp1_accessibility

In [ ]:
from asodesigner.features.RNaseH_features import rnaseh1_dict, compute_rnaseh1_score

best_window_start_krel = {
    'R4a_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 3, 18: 2, 19: 4, 20: 3, 21: 0, 22: 0, 25: 0},
    'R4b_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 1, 18: 3, 19: 1, 20: 3, 21: 0, 22: 0, 25: 0},
    'R7_krel': {10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 3, 17: 2, 18: 4, 19: 6, 20: 4, 21: 0, 22: 0, 25: 0},
}

for exp in ['R4a_krel', 'R4b_krel', 'R7_krel']:
    weights = rnaseh1_dict(exp)

    def score_row(row):
        length = len(row['Sequence'])
        pos = best_window_start_krel.get(exp, {}).get(length, 0)
        return compute_rnaseh1_score(row['Sequence'], weights, window_start=pos)


    col_name = f"RNaseH1_Krel_score_{exp}"

    # Yeast should have similar motifs, perhaps
    df_rb1[col_name] = df_rb1.apply(score_row, axis=1)
    df_tp53bp1[col_name] = df_tp53bp1.apply(score_row, axis=1)

RNaseH1_Krel_features_best = [f"RNaseH1_Krel_score_{exp}" for exp in ['R4a_krel', 'R4b_krel', 'R7_krel']]

In [ ]:
df_rb1_bak = df_rb1.copy()
df_tp53bp1_bak = df_tp53bp1.copy()

In [ ]:
# =========================
# External mRNA integration
# =========================
import pandas as pd
from pathlib import Path
import re

def _norm_rna_to_dna(seq: str) -> str:
    """Normalize RNA to DNA alphabet (U->T), uppercase, strip whitespace."""
    return str(seq).upper().replace('U', 'T').replace(' ', '').replace('\t', '').replace('\n', '')


def load_mrna_by_gene_from_files(files: list[str | Path],seq_column: str = "Original Transcript Sequence" ) -> dict[str, str]:
    """
    Load {Gene -> <seq_column>} from a manual list of CSV paths.
    - Expects columns: 'Gene' and <seq_column>
    - Returns DNA alphabet (A/C/G/T) after U->T via _norm_rna_to_dna
    - If multiple rows per gene: keeps the *longest* sequence
    """
    files = [Path(f) for f in files]
    rows = []
    for f in files:
        df = pd.read_csv(f, usecols=['Gene', seq_column])
        df[seq_column] = df[seq_column].map(_norm_rna_to_dna)
        # Keep only clean sequences
        df = df[df[seq_column].str.fullmatch(r'[ACGT]+', na=False)]
        rows.append(df)

    if not rows:
        return {}

    big = pd.concat(rows, ignore_index=True)
    big['len'] = big[seq_column].str.len()
    # Pick longest per gene
    chosen = big.sort_values(['Gene', 'len'], ascending=[True, False]).drop_duplicates('Gene')
    return dict(zip(chosen['Gene'], chosen[seq_column]))

# ---- Choose which mRNA to use for mRNA-based features (tAI/windows on mRNA, etc.) ----
def choose_preferred_mrna(gene_name: str, mrna_built_from_exons: str, gene_to_mrna_real: dict[str,str]) -> str:
    """
    Prefer the real (external) mRNA when available; otherwise fall back to exon-joined.
    Does NOT touch your genome->(mRNA/CDS) mappings or pre-mRNA flanks.
    """
    ext = gene_to_mrna_real.get(gene_name)
    return ext if ext else mrna_built_from_exons


In [ ]:
from pathlib import Path
import pandas as pd
from asodesigner.consts import *

DATA_mRNA_PATH = PROJECT_PATH / "scripts" / "data_genertion" / "cell_line_expression"

FILENAMES = [
    "ACH-000232_transcriptome.csv",
    "ACH-000463_transcriptome.csv",
    "ACH-000739_transcriptome.csv",
    "ACH-001086_transcriptome.csv",
    "ACH-001188_transcriptome.csv",
    "ACH-001328_transcriptome.csv",
]

EXTERNAL_MRNA_FILES = [DATA_mRNA_PATH / fn for fn in FILENAMES]

In [ ]:
d_orig = load_mrna_by_gene_from_files(
    [str(p) for p in EXTERNAL_MRNA_FILES],
    seq_column="Original Transcript sequence"
)

In [ ]:

import pandas as pd

df1 = pd.read_csv(DATA_mRNA_PATH / "ACH-000232_transcriptome.csv")
df2 = pd.read_csv(DATA_mRNA_PATH / "ACH-000463_transcriptome.csv")
df3 = pd.read_csv(DATA_mRNA_PATH / "ACH-000739_transcriptome.csv")
df4 = pd.read_csv(DATA_mRNA_PATH / "ACH-001086_transcriptome.csv")
df5 = pd.read_csv(DATA_mRNA_PATH / "ACH-001188_transcriptome.csv")
df6 = pd.read_csv(DATA_mRNA_PATH / "ACH-001328_transcriptome.csv")
transcript_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)
transcript_df = transcript_df.drop_duplicates()

In [ ]:
transcript_df = transcript_df.copy()

transcript_df.loc[:, "ref sequence"] = (transcript_df["Mutated Transcript sequence"].fillna(transcript_df["Original Transcript sequence"]))

In [ ]:
# Build CAI reference weights directly from top-N transcript sequences (no CDS extraction)

from asodesigner.features.cai import calc_CAI_weight  # make sure cai.py is importable

TOP_N   = 300
SEQ_COL = "ref sequence"
EXPR_COL = "expression_norm"

# Basic checks
assert EXPR_COL in transcript_df.columns, f"Missing '{EXPR_COL}' column"
assert SEQ_COL  in transcript_df.columns, f"Missing '{SEQ_COL}' column"

# 1) Pick top-N by expression_norm
ref_df = transcript_df.sort_values(EXPR_COL, ascending=False).head(TOP_N).copy()

# 2) Take their sequences as-is (mRNA with U's; calc_CAI_weight handles U->T internally)
reference_seqs = ref_df[SEQ_COL].dropna().astype(str).tolist()

# 3) Build CAI weights
weights_list, weights_flat = calc_CAI_weight(reference_seqs)

print(f"Built CAI weights from {len(reference_seqs)} transcript sequences (top {TOP_N} by {EXPR_COL}).")

In [ ]:
df_rb1_bak = df_rb1.copy()
df_tp53bp1_bak = df_tp53bp1.copy()

In [ ]:
from pathlib import Path
import pandas as pd

DATA_mRNA_PATH = PROJECT_PATH / "scripts" / "data_genertion" / "cell_line_expression"

FILENAMES = [
    "ACH-000232_transcriptome.csv",
    "ACH-000463_transcriptome.csv",
    "ACH-000739_transcriptome.csv",
    "ACH-001086_transcriptome.csv",
    "ACH-001188_transcriptome.csv",
    "ACH-001328_transcriptome.csv",
]

EXTERNAL_MRNA_FILES = [DATA_mRNA_PATH / fn for fn in FILENAMES]
missing = [p.name for p in EXTERNAL_MRNA_FILES if not p.exists()]
assert not missing, f"Missing files in {DATA_mRNA_PATH}: {missing}"

# 1) Load Original
try:
    d_orig = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Original Transcript sequence"
    )
except ValueError:
    # in case the S is capitalized in your headers
    d_orig = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Original Transcript Sequence"
    )

# 2) Load Mutated
try:
    d_mut = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Mutated Transcript sequence"
    )
except ValueError:
    d_mut = load_mrna_by_gene_from_files(
        [str(p) for p in EXTERNAL_MRNA_FILES],
        seq_column="Mutated Transcript Sequence"
    )

# 3) Prefer Mutated when available
gene_to_mrna_real = {**d_orig, **d_mut}

print(f"Loaded {len(gene_to_mrna_real)} real mRNA sequences (Gene -> mRNA).")



In [ ]:
from asodesigner.util import get_antisense
import numpy as np

# Column names
SENSE_LENGTH      = 'sense_length'         # Length of the ASO (nt)
SENSE_TYPE        = 'sense_type'           # exon / intron
CDS_SEQUENCE      = 'cds_sequence'         # CDS string (joined exons within CDS range)
IN_CODING_REGION  = 'in_coding_region'     # site is within CDS on a real exon

# Flank sizes
FLANK_SIZES_PREMRNA = [20, 30, 40, 50, 60, 70]
FLANK_SIZES_CDS     = [20, 30, 40, 50, 60, 70]

df_rb1[CDS_SEQUENCE]     = ""
df_rb1[IN_CODING_REGION] = False

for fs in FLANK_SIZES_PREMRNA:
    df_rb1[f"flank_sequence_{fs}"] = ""
for fs in FLANK_SIZES_CDS:
    df_rb1[f"local_coding_region_around_ASO_{fs}"] = ""

# ---- helpers (local to Part B) ----
def _to_str_seq(x) -> str:
    """
    Coerce sequence-like (list/np.array/Series) or string to a clean uppercase DNA string.
    Converts U->T and strips whitespace. Ensures slicing returns a plain string (avoids pandas iterable assignment).
    """
    if isinstance(x, str):
        s = x
    else:
        try:
            s = ''.join(list(x))
        except Exception:
            s = str(x)
    return s.replace(' ', '').replace('\t', '').replace('\n', '').replace('U', 'T').upper()

def _build_spliced_mrna_from_exons(pre_mrna: str, exon_indices):
    """
    Build exon-joined mRNA by concatenating exon slices out of pre_mrna.
    Keeps your original assumptions: pre_mrna corresponds to genomic strand and
    starts at exon_indices[0][0]; exon intervals are used directly.
    """
    if not exon_indices:
        return ""
    pre_genome_start = exon_indices[0][0]
    parts = []
    for exon_start, exon_end in exon_indices:
        pm_start = exon_start - pre_genome_start
        pm_end   = exon_end   - pre_genome_start
        parts.append(pre_mrna[pm_start:pm_end])
    return "".join(parts)

# Cache CDS per gene
gene_to_cds_info = {}

# ---- main loop ----
for index, row in df_rb1.iterrows():
    gene_name  = row[CANONICAL_GENE]
    locus_info = gene_to_data[gene_name]

    # Keep using your current pre-mRNA for flanks/exon-intron logic (coerced to clean string)
    pre_mrna  = _to_str_seq(locus_info.full_mrna)
    antisense = _to_str_seq(row[SEQUENCE])
    sense     = _to_str_seq(get_antisense(antisense))

    # Locate site on pre-mRNA
    idx = pre_mrna.find(sense)
    df_rb1.at[index, SENSE_LENGTH] = len(antisense)

    if idx != -1:
        # Genomic correction (kept as-is)
        genome_corrected_index = idx + locus_info.exon_indices[0][0]

        # pre-mRNA flanks (now using .at and guaranteed string slices)
        for fs in FLANK_SIZES_PREMRNA:
            flank_start = max(0, idx - fs)
            flank_end   = min(len(pre_mrna), idx + len(sense) + fs)
            flank_seq   = pre_mrna[flank_start:flank_end]
            df_rb1.at[index, f"flank_sequence_{fs}"] = flank_seq

        # Build CDS + genome->mRNA map (kept identical to your approach)
        if gene_name not in gene_to_cds_info:
            cds_seq = []  # build as list for speed, join at end
            genome_to_mrna_map = {}
            mrna_idx = 0
            for exon_start, exon_end in locus_info.exon_indices:
                for gpos in range(exon_start, exon_end):
                    if mrna_idx >= len(pre_mrna):
                        break
                    if locus_info.cds_start <= gpos <= locus_info.cds_end:
                        cds_seq.append(pre_mrna[mrna_idx])
                        genome_to_mrna_map[gpos] = len(cds_seq) - 1
                    mrna_idx += 1
            cds_seq = ''.join(cds_seq)
            gene_to_cds_info[gene_name] = (cds_seq, genome_to_mrna_map)
        else:
            cds_seq, genome_to_mrna_map = gene_to_cds_info[gene_name]

        # Save CDS
        df_rb1.at[index, CDS_SEQUENCE] = _to_str_seq(cds_seq)

        #  NEW: prefer real mRNA for mRNA-based features (fallback to exon-joined)
        mrna_built        = _build_spliced_mrna_from_exons(pre_mrna, locus_info.exon_indices)
        mrna_for_features = choose_preferred_mrna(gene_name, mrna_built, gene_to_mrna_real)

        # If within CDS, extract local CDS context (unchanged logic; .at + str)
        if (
            locus_info.cds_start <= genome_corrected_index <= locus_info.cds_end
            and genome_corrected_index in genome_to_mrna_map
        ):
            df_rb1.at[index, IN_CODING_REGION] = True
            cds_idx = genome_to_mrna_map[genome_corrected_index]
            for fs in FLANK_SIZES_CDS:
                start = max(0, cds_idx - fs)
                end   = min(len(cds_seq), cds_idx + len(sense) + fs)
                local_seq = cds_seq[start:end]
                df_rb1.at[index, f"local_coding_region_around_ASO_{fs}"] = _to_str_seq(local_seq)



In [ ]:
from asodesigner.util import get_antisense
import numpy as np

# Column names
SENSE_LENGTH      = 'sense_length'         # Length of the ASO (nt)
SENSE_TYPE        = 'sense_type'           # exon / intron
CDS_SEQUENCE      = 'cds_sequence'         # CDS string (joined exons within CDS range)
IN_CODING_REGION  = 'in_coding_region'     # site is within CDS on a real exon

# Flank sizes
FLANK_SIZES_PREMRNA = [20, 30, 40, 50, 60, 70]
FLANK_SIZES_CDS     = [20, 30, 40, 50, 60, 70]

df_tp53bp1[CDS_SEQUENCE]     = ""
df_tp53bp1[IN_CODING_REGION] = False

for fs in FLANK_SIZES_PREMRNA:
    df_tp53bp1[f"flank_sequence_{fs}"] = ""
for fs in FLANK_SIZES_CDS:
    df_tp53bp1[f"local_coding_region_around_ASO_{fs}"] = ""

# ---- helpers (local to Part B) ----
def _to_str_seq(x) -> str:
    """
    Coerce sequence-like (list/np.array/Series) or string to a clean uppercase DNA string.
    Converts U->T and strips whitespace. Ensures slicing returns a plain string (avoids pandas iterable assignment).
    """
    if isinstance(x, str):
        s = x
    else:
        try:
            s = ''.join(list(x))
        except Exception:
            s = str(x)
    return s.replace(' ', '').replace('\t', '').replace('\n', '').replace('U', 'T').upper()

def _build_spliced_mrna_from_exons(pre_mrna: str, exon_indices):
    """
    Build exon-joined mRNA by concatenating exon slices out of pre_mrna.
    Keeps your original assumptions: pre_mrna corresponds to genomic strand and
    starts at exon_indices[0][0]; exon intervals are used directly.
    """
    if not exon_indices:
        return ""
    pre_genome_start = exon_indices[0][0]
    parts = []
    for exon_start, exon_end in exon_indices:
        pm_start = exon_start - pre_genome_start
        pm_end   = exon_end   - pre_genome_start
        parts.append(pre_mrna[pm_start:pm_end])
    return "".join(parts)

# Cache CDS per gene
gene_to_cds_info = {}

# ---- main loop ----
for index, row in df_tp53bp1.iterrows():
    gene_name  = row[CANONICAL_GENE]
    locus_info = gene_to_data[gene_name]

    # Keep using your current pre-mRNA for flanks/exon-intron logic (coerced to clean string)
    pre_mrna  = _to_str_seq(locus_info.full_mrna)
    antisense = _to_str_seq(row[SEQUENCE])
    sense     = _to_str_seq(get_antisense(antisense))

    # Locate site on pre-mRNA
    idx = pre_mrna.find(sense)
    df_tp53bp1.at[index, SENSE_LENGTH] = len(antisense)

    if idx != -1:
        # Genomic correction (kept as-is)
        genome_corrected_index = idx + locus_info.exon_indices[0][0]

        # pre-mRNA flanks (now using .at and guaranteed string slices)
        for fs in FLANK_SIZES_PREMRNA:
            flank_start = max(0, idx - fs)
            flank_end   = min(len(pre_mrna), idx + len(sense) + fs)
            flank_seq   = pre_mrna[flank_start:flank_end]
            df_tp53bp1.at[index, f"flank_sequence_{fs}"] = flank_seq

        # Build CDS + genome->mRNA map (kept identical to your approach)
        if gene_name not in gene_to_cds_info:
            cds_seq = []  # build as list for speed, join at end
            genome_to_mrna_map = {}
            mrna_idx = 0
            for exon_start, exon_end in locus_info.exon_indices:
                for gpos in range(exon_start, exon_end):
                    if mrna_idx >= len(pre_mrna):
                        break
                    if locus_info.cds_start <= gpos <= locus_info.cds_end:
                        cds_seq.append(pre_mrna[mrna_idx])
                        genome_to_mrna_map[gpos] = len(cds_seq) - 1
                    mrna_idx += 1
            cds_seq = ''.join(cds_seq)
            gene_to_cds_info[gene_name] = (cds_seq, genome_to_mrna_map)
        else:
            cds_seq, genome_to_mrna_map = gene_to_cds_info[gene_name]

        # Save CDS
        df_tp53bp1.at[index, CDS_SEQUENCE] = _to_str_seq(cds_seq)

        #  NEW: prefer real mRNA for mRNA-based features (fallback to exon-joined)
        mrna_built        = _build_spliced_mrna_from_exons(pre_mrna, locus_info.exon_indices)
        mrna_for_features = choose_preferred_mrna(gene_name, mrna_built, gene_to_mrna_real)

        # If within CDS, extract local CDS context (unchanged logic; .at + str)
        if (
            locus_info.cds_start <= genome_corrected_index <= locus_info.cds_end
            and genome_corrected_index in genome_to_mrna_map
        ):
            df_tp53bp1.at[index, IN_CODING_REGION] = True
            cds_idx = genome_to_mrna_map[genome_corrected_index]
            for fs in FLANK_SIZES_CDS:
                start = max(0, cds_idx - fs)
                end   = min(len(cds_seq), cds_idx + len(sense) + fs)
                local_seq = cds_seq[start:end]
                df_tp53bp1.at[index, f"local_coding_region_around_ASO_{fs}"] = _to_str_seq(local_seq)



In [ ]:
df_tp53bp1

In [ ]:

from sklearn.preprocessing import StandardScaler

# Define flank sizes
CDS_WINDOWS = FLANK_SIZES_CDS

# Loop over each flank window size
for flank in CDS_WINDOWS:
    local_col = f"local_coding_region_around_ASO_{flank}"
    is_local_flag_col = f"region_is_local_{flank}"

    # Create the binary flag: 1 if local exists, 0 otherwise
    df_tp53bp1[is_local_flag_col] = df_tp53bp1[local_col].apply(
        lambda x: isinstance(x, str) and x.strip() != ""
    ).astype(int)

    # Create the binary flag: 1 if local exists, 0 otherwise
    df_rb1[is_local_flag_col] = df_rb1[local_col].apply(
        lambda x: isinstance(x, str) and x.strip() != ""
    ).astype(int)



In [ ]:
from asodesigner.features.cai import calc_CAI

# weights_list, weights_flat = calc_CAI_weight(reference_seqs)

for flank in CDS_WINDOWS:
    local_col = f"local_coding_region_around_ASO_{flank}"
    CAI_col   = f"CAI_score_{flank}_CDS"
    df_rb1[CAI_col] = (
        df_rb1[local_col].astype(str).apply(lambda s: calc_CAI(s, weights_flat))
    )
    df_tp53bp1[CAI_col] = (
        df_tp53bp1[local_col].astype(str).apply(lambda s: calc_CAI(s, weights_flat))
    )
df_rb1["CAI_score_global_CDS"] = (
    df_rb1["cds_sequence"].astype(str).apply(lambda s: calc_CAI(s, weights_flat))
)
df_tp53bp1["CAI_score_global_CDS"] = (
    df_tp53bp1["cds_sequence"].astype(str).apply(lambda s: calc_CAI(s, weights_flat))
)


CAI_list = [f"CAI_score_{flank}_CDS" for flank in CDS_WINDOWS] + ["CAI_score_global_CDS"]

In [ ]:
df_rb1_bak = df_rb1.copy()
df_tp53bp1_bak = df_tp53bp1.copy()

In [ ]:
df_tp53bp1 = df_tp53bp1_bak

In [ ]:
SENSE_START = 'sense_start'
SENSE_START_FROM_END = 'sense_start_from_end'
SENSE_LENGTH = 'sense_length'
SENSE_TYPE = 'sense_type'
SENSE_EXON = 'sense_exon'
SENSE_INTRON = 'sense_intron'
SENSE_UTR = 'sense_utr'

for index, row in df_tp53bp1.iterrows():
    gene_name = row[CANONICAL_GENE]
    locus_info = gene_to_data[gene_name]
    pre_mrna = locus_info.full_mrna
    antisense = row[SEQUENCE]
    idx = row[SENSE_START]
    df_tp53bp1.at[index, SENSE_START_FROM_END] = np.abs(
        locus_info.exon_indices[-1][1] - locus_info.cds_start - idx
    )
    if idx != -1:
        genome_corrected_index = idx + locus_info.cds_start
        found = False
        for exon_indices in locus_info.exon_indices:
            # print(exon[0], exon[1])
            if exon_indices[0] <= genome_corrected_index <= exon_indices[1]:
                df_tp53bp1.at[index, SENSE_TYPE] = 'exon'
                df_tp53bp1.at[index, SENSE_EXON] = 1
                found = True
                break
        for intron_indices in locus_info.intron_indices:
            # print(exon[0], exon[1])
            if intron_indices[0] <= genome_corrected_index <= intron_indices[1]:
                df_tp53bp1.at[index, SENSE_TYPE] = 'intron'
                df_tp53bp1.at[index, SENSE_INTRON] = 1
                found = True
                break
        for i, utr_indices in enumerate(locus_info.utr_indices):
                if utr_indices[0] <= genome_corrected_index <= utr_indices[1]:
                    df_tp53bp1.at[index, SENSE_TYPE] = 'utr'
                    df_tp53bp1.at[index, SENSE_UTR] = 1

                    found = True
                    break
    if not found:
        df_tp53bp1.loc[index, SENSE_TYPE] = 'intron'


In [ ]:
SENSE_START = 'sense_start'
SENSE_START_FROM_END = 'sense_start_from_end'
SENSE_LENGTH = 'sense_length'
SENSE_TYPE = 'sense_type'
SENSE_EXON = 'sense_exon'
SENSE_INTRON = 'sense_intron'
SENSE_UTR = 'sense_utr'

for index, row in df_rb1.iterrows():
    gene_name = row[CANONICAL_GENE]
    locus_info = gene_to_data[gene_name]
    pre_mrna = locus_info.full_mrna
    antisense = row[SEQUENCE]
    idx = row[SENSE_START]
    df_rb1.at[index, SENSE_START_FROM_END] = np.abs(
        locus_info.exon_indices[-1][1] - locus_info.cds_start - idx
    )
    if idx != -1:
        genome_corrected_index = idx + locus_info.cds_start
        found = False
        for exon_indices in locus_info.exon_indices:
            # print(exon[0], exon[1])
            if exon_indices[0] <= genome_corrected_index <= exon_indices[1]:
                df_rb1.at[index, SENSE_TYPE] = 'exon'
                df_rb1.at[index, SENSE_EXON] = 1
                found = True
                break
        for intron_indices in locus_info.intron_indices:
            # print(exon[0], exon[1])
            if intron_indices[0] <= genome_corrected_index <= intron_indices[1]:
                df_rb1.at[index, SENSE_TYPE] = 'intron'
                df_rb1.at[index, SENSE_INTRON] = 1
                found = True
                break
        for i, utr_indices in enumerate(locus_info.utr_indices):
                if utr_indices[0] <= genome_corrected_index <= utr_indices[1]:
                    df_rb1.at[index, SENSE_TYPE] = 'utr'
                    df_rb1.at[index, SENSE_UTR] = 1

                    found = True
                    break
    if not found:
        df_rb1.loc[index, SENSE_TYPE] = 'intron'


In [ ]:
df_tp53bp1_scores = model.predict(df_tp53bp1[selected_features])
df_rb1_scores = model.predict(df_rb1[selected_features])


In [ ]:
# Assuming you already have get_antisense(seq: str) -> str defined
# get_antisense, for some reason numba doesn't work well
tbl = str.maketrans("ACGTUacgtuNn", "TGCAAtgcaaNn")

df_tp53bp1["score"] = df_tp53bp1_scores
df_rb1["score"] = df_rb1_scores

df_tp53bp1["sense"] = df_tp53bp1[SEQUENCE].astype(str).str.translate(tbl).str[::-1]
(
    df_tp53bp1.assign(score=df_tp53bp1_scores)
    .sort_values("score", ascending=False)  # sort by score
    .to_csv("df_tp53bp1_scores_model2.csv", index=False)
)

df_rb1["sense"] = df_rb1[SEQUENCE].astype(str).str.translate(tbl).str[::-1]
(
    df_rb1.assign(score=df_rb1_scores)
    .sort_values("score", ascending=False)  # sort by score
    .to_csv("df_rb1_scores_model2.csv", index=False)
)

df_tp53bp1_sorted = df_tp53bp1.sort_values('score', ascending=False)
df_rb1_sorted = df_rb1.sort_values('score', ascending=False)

In [ ]:
df_tp53bp1_sorted.insert(0, "row_number", range(1, len(df_tp53bp1_sorted) + 1))
df_rb1_sorted.insert(0, "row_number", range(1, len(df_rb1_sorted) + 1))

In [ ]:
# pip install requests
import math, time, threading, urllib.parse, requests
from concurrent.futures import ThreadPoolExecutor, as_completed

UA = {"User-Agent": "python-requests gggenome/greedy"}


def _ggg_hits_leq_json(seq, k, db="hg38", timeout=60, retries=2):
    """Count hits with <=k mismatches via GGGenome JSON; fallback to CSV if needed."""
    s = str(seq).upper().replace("U", "T")
    q = urllib.parse.quote(s)
    url_json = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.json"
    url_csv = f"https://gggenome.dbcls.jp/{db}/{k}/nogap/{q}.csv?download"

    for attempt in range(retries + 1):
        try:
            r = requests.get(url_json, headers=UA, timeout=timeout)
            r.raise_for_status()
            try:
                data = r.json()
            except ValueError:
                raise RuntimeError("JSON parse failed")
            if isinstance(data, list):
                return len(data)
            if isinstance(data, dict):
                if "results" in data and isinstance(data["results"], list): return len(data["results"])
                if "hits" in data and isinstance(data["hits"], list):       return len(data["hits"])
                return sum(len(v) for v in data.values() if isinstance(v, list))
            return 10000
        except Exception:
            # greedy CSV fallback
            try:
                r2 = requests.get(url_csv, headers=UA, timeout=timeout)
                r2.raise_for_status()
                return sum(1 for ln in r2.text.splitlines() if ln and not ln.startswith("#"))
            except Exception:
                if attempt < retries:
                    continue
                return 10000
    return 0


def _d123_for_sequence(seq, db="hg38"):
    s = str(seq).upper().replace("U", "T")
    if not s:
        return (s, 0, 0, 0)
    L = len(s)
    k_allowed = max(0, math.floor(0.25 * L))  # GGGenome cap
    k0 = _ggg_hits_leq_json(s, 0, db=db)
    k1 = _ggg_hits_leq_json(s, 1, db=db) if k_allowed >= 1 else 0
    # k2 = _ggg_hits_leq_json(s, 2, db=db) if k_allowed >= 2 else 0
    # k3 = _ggg_hits_leq_json(s, 3, db=db) if k_allowed >= 3 else 0
    d1 = max(0, k1 - k0)
    # d2 = max(0, k2 - k1)
    # d3 = max(0, k3 - k2)
    return (s, d1, 0, 0, k0)


cache = {}


def add_gggenome_d123(main_df, seq_col="SEQUENCE", db="hg38", *, max_workers=32, print_every=10):
    seqs = (main_df[seq_col].astype(str).str.upper().str.replace("U", "T", regex=False))
    uniq = seqs.dropna().unique().tolist()
    N = len(uniq)
    print(f"[GGG] Unique sequences: {N} | db={db} | workers={max_workers}")

    global cache
    cache = {}
    lock = threading.Lock()
    t0 = time.perf_counter()
    errs = 0

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = {ex.submit(_d123_for_sequence, s, db): s for s in uniq}
        done = 0
        for fut in as_completed(futs):
            s = futs[fut]
            try:
                s_key, d1, d2, d3, d0 = fut.result()
            except Exception:
                d0 = d1 = d2 = d3 = 0
                with lock:
                    errs += 1
            with lock:
                cache[s] = (s, d1, d2, d3, d0)
                done += 1
                if (done == 1) or (done % print_every == 0) or (done == N):
                    elapsed = time.perf_counter() - t0
                    rps = done / elapsed if elapsed > 0 else 0.0
                    print(f"[GGG] {done}/{N} cached | ~{rps:.1f} seq/s | errors={errs}")

    main_df["ggg_d1"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[1])
    main_df["ggg_d0"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[4])
    # main_df["ggg_d2"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[2])
    # main_df["ggg_d3"] = seqs.map(lambda s: cache.get(s, (s, 0, 0, 0, 0))[3])

    print(f"[GGG] Finished in {time.perf_counter() - t0:.1f}s. Added columns: ggg_d1, ggg_d2, ggg_d3")
    return main_df




In [ ]:
# --- usage ---
# main_df = main_df[main_df[SENSE_START] != -1]
result = add_gggenome_d123(df_tp53bp1_sorted[:150], seq_col='sense', db="hg38", max_workers=1, print_every=2)

In [ ]:
result[['row_number', SEQUENCE, 'sense_start', 'ggg_d0', 'ggg_d1', 'sense_avg_accessibility', 'on_target_fold_openness_normalized40_15', 'at_skew', 'gc_content', 'sense_utr']]

In [ ]:
result_rb1 = add_gggenome_d123(df_rb1_sorted[:150], seq_col='sense', db="hg38", max_workers=1, print_every=2)

In [ ]:
r

In [ ]:
tresult_rb1[['row_number', SEQUENCE, 'sense_start', 'ggg_d0', 'ggg_d1', 'sense_avg_accessibility', 'on_target_fold_openness_normalized40_15', 'at_skew', 'gc_content', 'sense_utr']]